In [1]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import torch
import pandas as pd

In [2]:
def load_pulsar_dataset(adjust_ratio):
    df = pd.read_csv('data/chap02/pulsar_stars.csv')
    pulsars = np.asarray(df[df['target_class'] == 1])
    stars = np.asarray(df[df['target_class'] == 0])

    input_cnt, output_cnt = 8, 1

    star_cnt, pulsar_cnt = len(stars), len(pulsars)
    if adjust_ratio:
        data = np.zeros([2 * star_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        for n in range(star_cnt):
            data[star_cnt + n] = np.asarray(pulsars[n % pulsar_cnt], dtype='float32')
    else:
        data = np.zeros([star_cnt + pulsar_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        data[star_cnt:, :] = np.asarray(pulsars, dtype='float32')

    return data

In [21]:
data = load_pulsar_dataset(1)

In [22]:
data.shape

(32518, 9)

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data[:,:-1], data[:,-1], test_size=0.2, random_state=42)

In [24]:
trn = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
train_loader = DataLoader(trn, batch_size = 512, shuffle=True)

trn = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))
test_loader = DataLoader(trn, batch_size = 512, shuffle=True)

In [25]:
class NeuralNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.linear_1 = torch.nn.Linear(self.input_size, self.hidden_size, bias=True)
        
    def forward(self, input_tensor):
        linear1 = self.linear_1(input_tensor)
        out = torch.sigmoid(linear1)
        
        return out
        

In [26]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("Training on {}".format(DEVICE))

Training on cuda


In [27]:
model        = NeuralNet(8, 1).to(DEVICE)
optimizer    = torch.optim.SGD(model.parameters(), lr=0.01)
criterion    = torch.nn.BCELoss() 

In [28]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.squeeze(), target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


# ## 테스트하기

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += criterion(output.squeeze(), target).item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [29]:
EPOCHS = 30

for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/26014 (0%)]	Loss: 8.204283
Train Epoch: 1 [5120/26014 (20%)]	Loss: 2.637599
Train Epoch: 1 [10240/26014 (39%)]	Loss: 11.965713
Train Epoch: 1 [15360/26014 (59%)]	Loss: 1.930734
Train Epoch: 1 [20480/26014 (78%)]	Loss: 1.690676
Train Epoch: 1 [20700/26014 (98%)]	Loss: 1.609246
[1] Test Loss: 0.0037, Accuracy: 51.11%
Train Epoch: 2 [0/26014 (0%)]	Loss: 1.412377
Train Epoch: 2 [5120/26014 (20%)]	Loss: 1.439458
Train Epoch: 2 [10240/26014 (39%)]	Loss: 3.086080
Train Epoch: 2 [15360/26014 (59%)]	Loss: 5.236709
Train Epoch: 2 [20480/26014 (78%)]	Loss: 1.807706
Train Epoch: 2 [20700/26014 (98%)]	Loss: 1.796612
[2] Test Loss: 0.0030, Accuracy: 51.11%
Train Epoch: 3 [0/26014 (0%)]	Loss: 1.547927
Train Epoch: 3 [5120/26014 (20%)]	Loss: 4.394727
Train Epoch: 3 [10240/26014 (39%)]	Loss: 1.521451
Train Epoch: 3 [15360/26014 (59%)]	Loss: 1.367234
Train Epoch: 3 [20480/26014 (78%)]	Loss: 1.336256
Train Epoch: 3 [20700/26014 (98%)]	Loss: 0.968271
[3] Test Loss: 0.0014, Accuracy: 51.1